### The subscriber wants to optimize his communication costs by changing his current tariff plan. Three cellular operators sent their proposals: descriptions of all tariff plans were aggregated into a single document (tariff_plans.docx). There is also a download (Исходящие вызовы абонента.xlsx) of outgoing calls from a subscriber for the last 6 months.
##### It is necessary to investigate the nature of the subscriber’s outgoing calls and offer the optimal tariff, justifying your recommendations with analytical calculations.

In [979]:
import pandas as pd
import numpy as np
from datetime import datetime, date, time
from dateutil.parser import parse

In [980]:
ymd = lambda x: str(parse(x).date())      #format year-month-day
ym = lambda x: parse(x).strftime('%Y-%m') #format year-month
round_func = lambda x: round(x, 2)        #rounding

In [981]:
def to_minutes(x):
    time = parse(x).time()
    return time.minute + 1
def to_seconds(x):
    time = parse(x).time()
    seconds = time.hour*3600+time.minute*60 + time.second
    return seconds

In [982]:
def f_5(x):
    if x>5:
        return 5
    return x
def f_8(x):
    if x>8:
        return 8
    return x
def f_25(x):
    if x>25:
        return 25
    return x

In [983]:
df = pd.read_excel('Исходящие вызовы абонента.xlsx', engine='openpyxl')

In [984]:
df = df.rename(columns={'Дата звонка':'date', 'Длительность разговора':'length',
                                  'Оператор связи вызываемого абонента':'service'})

## MTS

In [985]:
mts = df[~(df.service == 'мтс')].copy()
mts = mts.drop('service', axis=1)
mts.head()

,date,length
0,15.12.2010 10:50:54,00:01:49
1,15.12.2010 11:50:42,00:00:57
2,30.11.2010 16:39:34,00:00:55
3,30.11.2010 16:39:34,00:00:55
4,01.12.2010 09:49:09,00:06:47


In [986]:
mts.date = mts['date'].map(ymd)

In [987]:
mts.length = mts['length'].map(to_seconds)

In [988]:
mts = mts.groupby('date').sum()
mts.head()

,length
date,
2010-01-07,75
2010-01-09,622
2010-01-10,843
2010-01-11,621
2010-01-12,2232


In [989]:
mts.length = mts.length/60

In [990]:
mts['0.90'] = mts['length'].map(f_5)

In [991]:
mts['0.05'] = mts.length - mts['0.90']

In [992]:
mts.head()

,length,0.90,0.05
date,,,
2010-01-07,1.250000,1.25,0.000000
2010-01-09,10.366667,5.00,5.366667
2010-01-10,14.050000,5.00,9.050000
2010-01-11,10.350000,5.00,5.350000
2010-01-12,37.200000,5.00,32.200000


In [993]:
mts['0.05'] = mts['0.05'].map(f_25)

In [994]:
mts['0.90'] = mts['0.90'] + (mts.length - mts['0.90'] - mts['0.05'])

In [995]:
mts.head()

,length,0.90,0.05
date,,,
2010-01-07,1.250000,1.25,0.000000
2010-01-09,10.366667,5.00,5.366667
2010-01-10,14.050000,5.00,9.050000
2010-01-11,10.350000,5.00,5.350000
2010-01-12,37.200000,12.20,25.000000


In [996]:
mts = mts.drop('length', axis=1)

In [997]:
mts['mts'] = 0.9*mts['0.90'] + 0.05*mts['0.05']
mts = mts.drop(['0.90', '0.05'], axis=1)

In [998]:
mts = mts.reset_index()

In [999]:
mts.head()

,date,mts
0,2010-01-07,1.125000
1,2010-01-09,4.768333
2,2010-01-10,4.952500
3,2010-01-11,4.767500
4,2010-01-12,12.230000


In [1000]:
mts.date = mts['date'].map(ym)

In [1001]:
mts.mts = mts['mts'].map(round_func)

In [1002]:
mts = mts.groupby('date').sum()
final_df = mts.sort_index(ascending = False).copy()

## Megafon

In [1003]:
megafon = df.copy()
megafon = megafon.drop('service', axis=1)
megafon.head()

,date,length
0,15.12.2010 10:50:54,00:01:49
1,15.12.2010 11:50:42,00:00:57
2,30.11.2010 16:39:34,00:00:55
3,30.11.2010 16:39:34,00:00:55
4,01.12.2010 09:49:09,00:06:47


In [1004]:
megafon.date = megafon['date'].map(ym)

In [1005]:
megafon.length = megafon['length'].map(to_minutes)

In [1006]:
megafon['rub'] = pd.Series(np.ones(megafon.shape[0]))

In [1007]:
megafon['0.33'] = megafon.length - megafon.rub

In [1008]:
megafon['megafon'] = 1*megafon.rub + 0.33*megafon['0.33']

In [1009]:
megafon = megafon.drop(['length', 'rub', '0.33'], axis = 1)

In [1010]:
megafon = megafon.groupby('date').sum()

In [1011]:
final_df = final_df.join(megafon)

## Beeline(1)

In [1012]:
bilain_1 = df.copy()
bilain_1 = bilain_1.drop('service', axis=1)
bilain_1.head()

,date,length
0,15.12.2010 10:50:54,00:01:49
1,15.12.2010 11:50:42,00:00:57
2,30.11.2010 16:39:34,00:00:55
3,30.11.2010 16:39:34,00:00:55
4,01.12.2010 09:49:09,00:06:47


In [1013]:
bilain_1.date = bilain_1['date'].map(ym)
bilain_1.length = bilain_1['length'].map(to_minutes)

In [1014]:
bilain_1['1.50'] = pd.Series(np.ones(bilain_1.shape[0]))

In [1015]:
bilain_1['0.50'] = bilain_1.length - bilain_1['1.50']

In [1016]:
bilain_1['0.50'] = bilain_1['0.50'].map(f_8)

In [1017]:
bilain_1['1.00'] = bilain_1.length - bilain_1['1.50'] - bilain_1['0.50']

In [1018]:
bilain_1['bilain_1'] = 1.5*bilain_1['1.50'] + 0.5*bilain_1['0.50'] + bilain_1['1.00']

In [1019]:
bilain_1 = bilain_1.drop(['length', '1.50', '0.50', '1.00'], axis = 1)

In [1020]:
bilain_1 = bilain_1.groupby('date').sum()

In [1021]:
final_df = final_df.join(bilain_1)

## Beeline(2)

In [1022]:
bilain_2 = df.copy()
bilain_2 = bilain_2.drop('service', axis=1)
bilain_2.head()

,date,length
0,15.12.2010 10:50:54,00:01:49
1,15.12.2010 11:50:42,00:00:57
2,30.11.2010 16:39:34,00:00:55
3,30.11.2010 16:39:34,00:00:55
4,01.12.2010 09:49:09,00:06:47


In [1023]:
bilain_2.date = bilain_2['date'].map(ymd)
bilain_2.length = bilain_2['length'].map(to_seconds)

In [1024]:
bilain_2 = bilain_2.groupby('date').sum()

In [1025]:
bilain_2.length = bilain_2.length/60

In [1026]:
bilain_2['3.95'] = bilain_2['length'].map(f_5)

In [1027]:
bilain_2['0.40'] = bilain_2['length'] - bilain_2['3.95']

In [1028]:
bilain_2['bilain_2'] = bilain_2['3.95']*3.95 + bilain_2['0.40']*0.4

In [1029]:
bilain_2 = bilain_2.reset_index()

In [1030]:
bilain_2.date = bilain_2['date'].map(ym)

In [1031]:
bilain_2 = bilain_2.drop(['length', '3.95', '0.40'], axis = 1)

In [1032]:
bilain_2 = bilain_2.groupby('date').sum()

In [1033]:
final_df = final_df.join(bilain_2)

In [1034]:
final_df

,mts,megafon,bilain_1,bilain_2
date,,,,
2010-12,43.24,131.63,201.5,241.130000
2010-11,189.70,311.74,510.5,564.496667
2010-10,118.16,377.99,577.5,531.495833
2010-09,88.48,284.33,428.5,426.518333
2010-08,80.76,195.44,296.0,318.465000
2010-07,44.46,136.04,210.5,226.539167
2010-06,11.82,35.60,53.5,66.645833
2010-05,11.89,23.97,36.0,55.219167
2010-04,12.45,42.23,64.0,61.004167


### Conclusion: if the subscriber used the MTS tariff, its costs each month would be much lower compared to other tariffs.
### Recommended operator: MTS